## Libraries

In [32]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
import json
from langchain.prompts import PromptTemplate

In [33]:
Model = Ollama(model = "gemma2")

In [34]:
vectors = FAISS.load_local("../VectorStores/nomic/AllPrinters", OllamaEmbeddings(model =  "nomic-embed-text"), allow_dangerous_deserialization= True)
retriever = vectors.as_retriever()

In [36]:
def combineDocs(docs):
    context = "\n\n".join(f'Document [Metadata :  Source : {doc.metadata.get("source")} \nContent : \n{doc.page_content} ]' for doc in docs)
    return context

In [5]:
chatHistory = "\n"

latestPrompt = ""

template = """
You are a helpful assistant. You are given some text, conversation history and a question. 
Answer the question based on the information given in the text (make use of the metadata to determine which text documents to use for the printer mentioned in the question) or based on the conversation if needed
Provide the source of your answer whether it's from a document (name the document) or the previous conversation

## Text ##
{context}

## Conversation ##
{conversation}
\n
Question : {question}
Answer: 

"""

prompt = PromptTemplate.from_template(template)

def chat(question): 
    
    global chatHistory
    global latestPrompt
    # Retrieve docs from vector DB
    #docs = vectors.similarity_search(question, k = 4)
    #contextString = combineDocs2(docs)
    contextString = combineDocs(retriever.invoke(question))
    
    query = prompt.format(conversation = chatHistory, context = contextString, question = question)
    latestPrompt = query
    response = Model.invoke(query)
    
    chatHistory = chatHistory + "\nQuestion : " + question + "\nAnswer : " + response

    return response


In [ ]:
chat("what are the various parts of a printer in a Deskjet3700")

In [ ]:
chat("can you give me the previous answer as a paragraph")

In [ ]:
print(chatHistory)

In [6]:
chat("what are the steps to print a photo on photo paper (Windows) using deskjet 2623")

'Here\'s how to print a photo on photo paper using your HP Deskjet 2623 on Windows, based on the provided manual excerpts:\n\n**1. Prepare the Printer:**\n\n* **Swap Cartridges:** Remove the black print cartridge and replace it with the photo cartridge. The manual states that you should store the black cartridge in an airtight container when not in use.\n* **Align Cartridges:**  If this is a new cartridge, align them according to your printer\'s instructions. Use plain paper for alignment to save your photo paper.\n\n**2. Load Photo Paper:**\n\n* Load the photo paper into the input tray, glossy-side down.\n\n**3. Print Settings (Windows):**\n\n* **Open Print Dialog:** In your software, go to "File" and select "Print".\n* **Select Printer:** Ensure your HP Deskjet 2623 is chosen as the printer.\n* **Properties/Settings:** Click the button that opens the Properties dialog box (it might be labeled differently depending on your software).\n* **Paper Type:** In the "Setup" tab, select "HP P

In [ ]:
print(retriever.invoke("what are the steps to print a photo on photo paper (Windows) using deskjet 2600 (aka deskjet 2623)"))

## Using Message Arrays

In [37]:
import ollama

In [43]:
history = [' ', ' ', ' ', ' ', ' ', ' '] 
def chat2(question) : 
    
    global history

    contextString = combineDocs(retriever.invoke(question))
    systemMessage = """ 
    You are a helpful assistant. You are given conversation history and a question with some context from the user. 
    Answer the question based on the information given in that context or based on the conversation history if needed."
    
    """
    message=[
                {
                    'role': 'system',
                    'content': systemMessage,
                },
                {
                    'role': 'user',
                    'content': history[-6],
                },
                {
                    'role': 'assistant',
                    'content': history[-5],
                },
                {
                    'role': 'user',
                    'content': history[-4],
                },
                {
                    'role': 'assistant',
                    'content': history[-3],
                },
                {
                    'role': 'user',
                    'content': history[-2],
                },
                {
                    'role': 'assistant',
                    'content': history[-1],
                },
                {
                    'role': 'user',
                    'content': "Here is the context : \n" + contextString,
                },{
                    'role': 'user',
                    'content': question,
                },
        ]
    
    response = ollama.chat(model = "Gemma2", messages = message)
    answer = response['message']['content']
    history.append(question)
    history.append(answer)
    
    print(answer)
    

In [44]:
chat2("what are the various parts of a printer in a Deskjet3700")

Based on the provided document "[Metadata :  Source : ..\Data\multiplePrinterBooks\HP - Deskjet-3700-Printer.pdf 
Content :  
HP DeskJet 3700 All-in-One series]", here are the parts of a Deskjet 3700 printer:

1. **Paper-width guide**
2. **Input tray extender**
3. **Input tray**
4. **Input tray shield**
5. **Scanner**
6. **Scan path**
7. **Scan light**
8. **Control panel**
9. **Cartridge access door**
10. **Cartridges**
11. **Output tray**
12. **Output tray extender (also referred to as the tray extender)**
13. **Paper catch**
14. **Power connection**
15. **USB port** 


Let me know if you'd like more details about any specific part!


In [45]:
chat2("Can you give me the previous answer as a paragraph?")

Based on the provided document, the HP DeskJet 3700 printer consists of several parts:  The input tray system includes a paper-width guide, input tray extender, input tray, and input tray shield. The printer also features a scanner with a scan path and scan light.  Users interact with the device through the control panel. The cartridge access door houses the ink cartridges. Finally, the output tray, including an output tray extender, and a paper catch facilitate printing. Additionally, the printer has power and USB connection ports. 





In [46]:
chat2("how can I load the multipurpose tray in e877 printer series")

Here's how to load the multipurpose tray (Tray 1) on your HP Color LaserJet E877 series printer:

**1. Access the Tray:**

* Locate the multipurpose tray on the front of your printer. It's usually labeled "Tray 1" and may be slightly smaller than the main paper trays.

**2. Open the Tray:**

* Gently pull the tray out from the printer until it stops.

**3. Adjust the Paper Guides:**

* **Width Guides:** Slide the width guides inward to match the width of your paper. Make sure the edges of the paper are snug against the guides.
* **Length Guide:**  Slide the length guide to the appropriate position for the size of paper you're loading.

**4. Load Your Paper:**

* Place your paper face down into the tray, making sure it's stacked evenly and not curled or bent. 
* Do not overload the tray. The maximum capacity is 100 sheets of 75 grams per square meter (20 lb) paper or 10 envelopes.

**5. Close the Tray:**

* Gently push the tray back into the printer until it clicks into place.

**6. Sel

In [47]:
retriever.invoke("how can I load the multipurpose tray in e877 printer series")

[Document(metadata={'source': '..\\Data\\multiplePrinterBooks\\HP - Deskjet-3700-Printer.pdf', 'file_path': '..\\Data\\multiplePrinterBooks\\HP - Deskjet-3700-Printer.pdf', 'page': 16, 'total_pages': 118, 'format': 'PDF 1.6', 'title': 'HP DeskJet 3700 All-in-One series User Guide – ENWW', 'author': 'HP IWS Design Customer Education', 'subject': '', 'keywords': 'Edition 1, 9/2016', 'creator': 'AH XSL Formatter V6.2 MR6 for Windows (x64) : 6.2.8.19325 (2015/01/06 12:40JST)', 'producer': 'Antenna House PDF Output Library 6.2.640 (Windows (x64))', 'creationDate': "D:20160524165601+08'00'", 'modDate': "D:20160525102417+08'00'", 'trapped': ''}, page_content='4.\nPull out the output tray and the output tray extender, and then lift the paper catch.\nENWW\nLoad media\n13\n'),
 Document(metadata={'source': '..\\Data\\multiplePrinterBooks\\hp-laserjet-Pro-m501-m506-m527-troubleshooting-manual.pdf', 'file_path': '..\\Data\\multiplePrinterBooks\\hp-laserjet-Pro-m501-m506-m527-troubleshooting-manual